### Лабораторная работа №2. Реализация глубокой нейронной сет

In [18]:
import os
import math
import tarfile
from PIL import Image

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tar_path = 'notMNIST_large.tar.gz'
tar_general_dir = 'notMNIST_large'

**Задание 1.** Реализуйте полносвязную нейронную сеть с помощью библиотеки Tensor Flow. В качестве алгоритма оптимизации можно использовать, например, стохастический градиент (Stochastic Gradient Descent, SGD). Определите количество скрытых слоев от 1 до 5, количество нейронов в каждом из слоев до нескольких сотен, а также их функции активации (кусочно-линейная, сигмоидная, гиперболический тангенс и т.д.).

Загрузим все изображения из архива

In [19]:
with tarfile.open(tar_path, 'r:gz') as tar:
    tar.extractall(path='.')

Просеем выборку и удалим из нее все битые файлы. Они помешают работе генератора в дальнейшем.

In [ ]:
for root, dirs, files in os.walk(tar_general_dir):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with Image.open(file_path) as img:
                img.verify()
        except:
            print(f"File removed: {file_path}")
            os.remove(file_path)

Спроектируем модель с пятью слоями

In [37]:
input_shape = (28, 28, 1)
num_classes = 10

model = Sequential()
model.add(Flatten(input_shape=input_shape))  # Преобразование 2D изображений в 1D вектор

model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='sigmoid'))

model.add(Dense(num_classes, activation='softmax'))  # Выходной слой с softmax активацией для классификации

Скомпилируем модель и получим информацию о ней

In [ ]:
model.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Подготовим генераторы. Генератор тренировочных данных.

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.05)

train_generator = datagen.flow_from_directory(
    tar_general_dir,
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',
    subset="training"
)

Генератор тестовых данных.

In [ ]:
test_generator = datagen.flow_from_directory(
    tar_general_dir,
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',
    subset="validation"
)

Обучим модель и получим ее точность на тестовых данных

In [ ]:
model.fit(train_generator, epochs=10)
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.4f}')

**Задание 2.** Как улучшилась точность классификатора по сравнению с логистической регрессией?

Можно пронаблюдать значительный рост точности классификации. Связано это с использованием нескольких слоев для построения полносвязной нейронной сети, вместо единственной классифицирующей модели (в прошлой лабораторной это была логистическая регрессия)

**Задание 3.** Используйте регуляризацию и метод сброса нейронов (dropout) для борьбы с переобучением. Как улучшилось качество классификации?

Сутью метода `Dropout` является добавление слоя сброса после некоторого полносвязного слоя. Слой сброса с некоторой вероятностью должен отключать случаюные нейроны во время каждой итерации обучения, что помогает предотвратить переобучение.

`Регуляризация` добавлена к последнему полносвязному слою. Она выставляет штраф за большие веса, что также помогает предотвратить переобучение.

In [42]:
input_shape = (28, 28, 1)

model = Sequential()
model.add(Flatten(input_shape=input_shape))

model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01)))

Скомпилируем и обучим новую модель, а также проведем ее оценку.

In [ ]:
model.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10)
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.4f}')

Как видно, точность осталась примерно на тех же значениях, что и была. Это означает, что переобучение на десяти эпохах для данного набора данных отсутсвует.

**Задание 4.** Воспользуйтесь динамически изменяемой скоростью обучения (learning rate). Наилучшая точность, достигнутая с помощью данной модели составляет 97.1%. Какую точность демонстрирует Ваша реализованная модель?


Для внедрения динамической скорости обучения необходимо определить `Learning Rate Scheduler` и поменять оптимизатор на `adam`.

Определим функцию `callback`

In [44]:
def scheduler(epoch, lr):
    if epoch < 5:
        return float(lr)
    else:
        return float(lr * math.exp(-0.1))

Спроектируем модель

In [45]:
input_shape = (28, 28, 1)

model = Sequential()
model.add(Flatten(input_shape=input_shape))

model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='tanh'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01)))

Скомпилируем и обучим новую модель, а также проведем ее оценку.

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, callbacks=[callback])
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.4f}')

Внедрение динамической скорости обучения несколько повысило точность классификаци, однако уровня 97% достигнуть не удалось. Дальнейшее повышение точности модели, очевидно, возможно лишь через увеличение количества эпох обучения.